In [1]:
! unzip /content/drive/MyDrive/DS_shuffled.zip -d /content/

Archive:  /content/drive/MyDrive/DS_shuffled.zip
   creating: /content/content/DeepSig_shuffled/
  inflating: /content/content/DeepSig_shuffled/datasets.desktop  
  inflating: /content/content/DeepSig_shuffled/DeepSig_XYZ_OSC.0001_1024_Shuffled.hdf5  
  inflating: /content/content/DeepSig_shuffled/classes-fixed.json  
  inflating: /content/content/DeepSig_shuffled/LICENSE.TXT  
  inflating: /content/content/DeepSig_shuffled/classes.txt  
  inflating: /content/content/DeepSig_shuffled/classes-fixed.txt  


In [14]:
#code block for loading in DeepSig data from hdf5 file
import tensorflow as tf
import h5py
import numpy as np
import json
import pandas as pd
import math

# Open the dataset
hdf5_file = h5py.File("/content/content/DeepSig_shuffled/DeepSig_XYZ_OSC.0001_1024_Shuffled.hdf5",  'r')
# Load the modulation classes. You can also copy and paste the content of classes-fixed.txt.
modulation_classes = json.load(open("/content/content/DeepSig_shuffled/classes-fixed.json", 'r'))
#List groups of the hdf5 file
list(hdf5_file.keys())

['X', 'Y', 'Z']

In [15]:
#Grabbing an eighth of the total data samples, dividing it into splits
data = hdf5_file['X'][0::8]
modulation_onehot = hdf5_file['Y'][0::8]
snr = hdf5_file['Z'][0::8]

#converting onehot back to normal
mod_array = np.zeros(len(modulation_onehot))
i = 0
for x in modulation_onehot:
    mod_array[i] = np.where(x==1)[0][0]
    i = i + 1

In [16]:
from sklearn.utils import shuffle
data, mod_array, snr = shuffle(data, mod_array, snr, random_state=0)

print(data)
print(modulation_onehot)
print(snr)

[[[-0.48748383  0.289871  ]
  [-0.10943886  0.10092989]
  [ 0.1539666   0.01478273]
  ...
  [ 0.8174222  -0.86155516]
  [ 0.90480685 -0.50140667]
  [ 0.66664314 -0.4372452 ]]

 [[ 0.5737007   0.12489015]
  [ 0.5311588   0.41893125]
  [ 0.44867843  0.684345  ]
  ...
  [ 0.8114136   0.06359135]
  [ 0.6394054   0.20864716]
  [ 0.44694647  0.29225376]]

 [[ 0.16258396 -0.3454956 ]
  [-0.0560893  -0.9101686 ]
  [-1.2954125  -0.10088739]
  ...
  [ 0.62307304 -0.81322855]
  [ 0.38167125  0.46994394]
  [-1.0193765   0.48680335]]

 ...

 [[-0.8882095   0.8573086 ]
  [-0.7635585   0.9484377 ]
  [-0.6620279   0.9982166 ]
  ...
  [ 0.01837325  0.9636689 ]
  [-0.19691336  0.8765645 ]
  [-0.35165292  0.7468494 ]]

 [[-1.1415536   0.30011955]
  [ 0.03811142  0.622609  ]
  [-0.6100388  -0.01606742]
  ...
  [-0.3258155  -0.6628931 ]
  [ 1.0614647  -1.006968  ]
  [ 0.3346123   0.40247998]]

 [[ 1.0012457   0.04313583]
  [ 0.99243045  0.21210092]
  [ 0.94953865  0.42650628]
  ...
  [-0.28701356  0.947195

In [17]:
trn_ind = math.floor(len(data) * .8)
val_ind = math.floor(len(data) * .1)
#test_ind = len(data) - trn_ind - val_ind

training_set, training_label, training_snr = data[:trn_ind:], mod_array[:trn_ind:], snr[:trn_ind:]
validation_set, validation_label, validation_snr = data[trn_ind:trn_ind+val_ind:], mod_array[trn_ind:trn_ind+val_ind:], snr[trn_ind:trn_ind+val_ind:]
test_set, test_label, test_snr = data[trn_ind+val_ind::], mod_array[trn_ind+val_ind::], snr[trn_ind+val_ind::]

tr_dataset = tf.data.Dataset.from_tensor_slices((training_set, training_label))
val_dataset = tf.data.Dataset.from_tensor_slices((validation_set, validation_label))
test_dataset = tf.data.Dataset.from_tensor_slices((test_set, test_label))

In [18]:
batch_size = 1024
buf_size = 26624
drop_rem = True

tr_dataset = tr_dataset.shuffle(buffer_size=buf_size)
tr_dataset = tr_dataset.batch(batch_size, drop_remainder=drop_rem)

val_dataset = val_dataset.shuffle(buffer_size=1000)
val_dataset = val_dataset.batch(batch_size=500, drop_remainder=True)

test_dataset = test_dataset.shuffle(buffer_size=1000)
test_dataset = test_dataset.batch(batch_size=500, drop_remainder=True)

In [19]:
#model variables
img_width, img_height, img_channels = (1024, 2, 1)
opt = 'Adam'
lss = 'sparse_categorical_crossentropy'


model = tf.keras.models.Sequential([
      tf.keras.layers.Conv1D(64, 3, activation='relu', 
                  input_shape=(img_width, img_height)),
      tf.keras.layers.MaxPooling1D(2, 2),
      tf.keras.layers.Conv1D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling1D(2),
      tf.keras.layers.Conv1D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling1D(2),
      tf.keras.layers.Conv1D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling1D(2),
      tf.keras.layers.Conv1D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling1D(2),
      tf.keras.layers.Conv1D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling1D(2),
      tf.keras.layers.Conv1D(64, 3, activation='relu'),
      tf.keras.layers.MaxPooling1D(2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation=tf.nn.selu),
      tf.keras.layers.Dense(128, activation=tf.nn.selu),
      tf.keras.layers.Dense(24, activation=tf.nn.softmax)
    ])

model.compile(optimizer=opt, loss=lss, metrics=['accuracy'])

In [20]:
model.fit(tr_dataset, epochs=10, validation_data=val_dataset, verbose=1)

Epoch 1/10
249/249 [==============================] - 792s 3s/step - loss: 2.3409 - accuracy: 0.2177 - val_loss: 1.9175 - val_accuracy: 0.3229
Epoch 2/10
249/249 [==============================] - 751s 3s/step - loss: 1.8559 - accuracy: 0.3375 - val_loss: 1.7924 - val_accuracy: 0.3517
Epoch 3/10
249/249 [==============================] - 719s 3s/step - loss: 1.7839 - accuracy: 0.3589 - val_loss: 1.7342 - val_accuracy: 0.3759
Epoch 4/10
249/249 [==============================] - 733s 3s/step - loss: 1.7460 - accuracy: 0.3706 - val_loss: 1.7035 - val_accuracy: 0.3857
Epoch 5/10
249/249 [==============================] - 750s 3s/step - loss: 1.7217 - accuracy: 0.3791 - val_loss: 1.7047 - val_accuracy: 0.3824
Epoch 6/10
249/249 [==============================] - 775s 3s/step - loss: 1.7001 - accuracy: 0.3859 - val_loss: 1.6981 - val_accuracy: 0.3806
Epoch 7/10
249/249 [==============================] - 764s 3s/step - loss: 1.6881 - accuracy: 0.3906 - val_loss: 1.6698 - val_accuracy: 0.3966

In [21]:
results = model.predict(test_set[:100:])

In [25]:
predictions = [p.argmax() for p in results]

print(len(predictions))
print(*[(c, a, c == a) for c, a in zip(test_label[:100:], predictions)], sep="\n")

100
(1.0, 1, True)
(17.0, 18, False)
(13.0, 8, False)
(13.0, 13, True)
(22.0, 22, True)
(2.0, 2, True)
(5.0, 6, False)
(15.0, 4, False)
(3.0, 3, True)
(21.0, 21, True)
(22.0, 22, True)
(12.0, 13, False)
(12.0, 0, False)
(13.0, 8, False)
(7.0, 8, False)
(17.0, 18, False)
(2.0, 2, True)
(17.0, 17, True)
(2.0, 2, True)
(3.0, 3, True)
(12.0, 22, False)
(0.0, 0, True)
(13.0, 21, False)
(9.0, 13, False)
(22.0, 22, True)
(7.0, 6, False)
(1.0, 1, True)
(6.0, 22, False)
(23.0, 22, False)
(19.0, 20, False)
(16.0, 13, False)
(12.0, 21, False)
(23.0, 23, True)
(20.0, 20, True)
(12.0, 8, False)
(7.0, 5, False)
(21.0, 21, True)
(10.0, 3, False)
(17.0, 17, True)
(6.0, 6, True)
(10.0, 13, False)
(8.0, 13, False)
(7.0, 13, False)
(4.0, 4, True)
(3.0, 3, True)
(23.0, 23, True)
(6.0, 17, False)
(14.0, 14, True)
(10.0, 13, False)
(21.0, 21, True)
(23.0, 9, False)
(16.0, 13, False)
(11.0, 13, False)
(15.0, 9, False)
(3.0, 3, True)
(3.0, 22, False)
(18.0, 22, False)
(16.0, 13, False)
(10.0, 13, False)
(11.0